## Community Detection Clustering in Complex Networks using Gumbel Softmax

### Authors: Visaj Nirav Shah (201801016) and Vyom Saraf (201801062)
#### Mentor: Prof. Mukesh Tiwari

*Parts of this code were adopted from the codebase of D. B. Acharya and H. Zhang, “Community detection clustering via gumbel softmax,” SN Computer Science, vol. 1, no. 5, aug 2020. [Online]. Available: https://doi.org/10.1007%2Fs42979-020-00264-2*

Note:- In case the cells are hidden, please click on the arrows on the left hand-side to expand that particular section.

**Structure of the Notebook**

- Section 1: Mathematics of Datasets
- Section 2: Normal Clustering
- Section 3: Custom Threshold Clustering (Multiple Clustering)
- Section 4: Determining Ideal Number of Clusters
- Section 5: Cluster Network Analysis

**Importing the Required Libraries**

In [ ]:
import math
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.animation as animation
import networkx as nx
import numpy as np
import pandas as pd

from collections import deque
from itertools import product
from scipy import sparse
from scipy.linalg import eig
from sklearn import metrics
from sklearn.metrics.cluster import normalized_mutual_info_score,adjusted_rand_score

device = torch.device('cpu')

# Section 1: Mathematics of Datasets

### Zachary's Karate Club Dataset

In [ ]:
# Creating a Networkx Graph

G = nx.karate_club_graph()

In [ ]:
# Number of nodes and edges

print("The network has " + str(G.number_of_nodes()) + " nodes.")
print("The network has " + str(G.number_of_edges()) + " edges.")

In [ ]:
# Adjacency Matrix

adj_mat = nx.adjacency_matrix(G).toarray()
adj_mat

In [ ]:
# Degree of nodes

degree = {}
for i in range(adj_mat.shape[0]):
    degree[i] = sum(adj_mat[i])
degree

In [ ]:
# Mean Degree

sumD = 0
for i in degree.keys():
    sumD += degree[i]
meanD = sumD / adj_mat.shape[0]
meanD

In [ ]:
# Density

density = meanD / (adj_mat.shape[0] - 1)
density

In [ ]:
# Degree Distribution plot

degree_list = sorted(set(degree.values()))
count_list = [list(degree.values()).count(i) for i in degree_list]

plt.scatter(degree_list, count_list)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Degree')
plt.ylabel('Number of Nodes')
plt.title('Logarithmic Scatter Plot - Degree Distribution')
plt.savefig('Karate Degree Distribution')

In [ ]:
# graph Laplacian

graph_laplacian = nx.laplacian_matrix(G).toarray()
graph_laplacian

In [ ]:
# Betweenness

between = nx.betweenness_centrality(G)
between

In [ ]:
# Closeness

close = nx.closeness_centrality(G)
close

In [ ]:
# Eigenvector centrality

eigenvector_centrality = nx.eigenvector_centrality(G)
eigenvector_centrality

In [ ]:
desc = {}
desc['Quantity'] = ['Nodes', 'Edges', 'Mean Degree', 'Density']
desc['Value'] = [G.number_of_nodes(), G.number_of_edges(), meanD, density]

desc_df = pd.DataFrame(desc)
desc_df

### Facebook Page-Page Networks Dataset of Artists’ Pages

In [ ]:
# Creating a Networkx Graph

network1 = pd.read_csv(r"artist_edges.csv")
edge_list = []
for i in range(819306):
    edge_list.append((network1.iloc[i]['node_1'], network1.iloc[i]['node_2']))

G1 = nx.Graph()
G1.add_edges_from(edge_list)

In [ ]:
# Number of nodes and edges

print("The network has " + str(G1.number_of_nodes()) + " nodes.")
print("The network has " + str(G1.number_of_edges()) + " edges.")

In [ ]:
# Adjacency Matrix

adj_mat1 = nx.adjacency_matrix(G1).toarray()
adj_mat1

In [ ]:
# Degree of nodes

degree1 = {}
for i in range(adj_mat1.shape[0]):
    degree1[i] = sum(adj_mat1[i])

In [ ]:
# Mean Degree

sumD = 0
for i in degree1.keys():
    sumD += degree1[i]
meanD1 = sumD / adj_mat1.shape[0]
meanD1

In [ ]:
# Density

density1 = meanD1 / (adj_mat1.shape[0] - 1)
density1

In [ ]:
# Degree Distribution plot

degree_list1 = sorted(set(degree1.values()))
count_list1 = [list(degree1.values()).count(i) for i in degree_list1]

plt.scatter(degree_list1, count_list1)
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Degree')
plt.ylabel('Number of Nodes')
plt.title('Logarithmic Scatter Plot - Degree Distribution')
plt.savefig('Facebook Degree Distribution')

In [ ]:
# graph Laplacian

graph_laplacian = nx.laplacian_matrix(G1).toarray()
graph_laplacian

In [ ]:
# Betweenness

between1 = nx.betweenness_centrality(G1)
between1

In [ ]:
# Closeness

close1 = nx.closeness_centrality(G1)
close1

In [ ]:
# Eigenvector centrality

eigenvector_centrality = nx.eigenvector_centrality(G1)
eigenvector_centrality

In [ ]:
desc1 = {}
desc1['Quantity'] = ['Nodes', 'Edges', 'Mean Degree', 'Density']
desc1['Value'] = [G1.number_of_nodes(), G1.number_of_edges(), meanD1, density1]

desc1_df = pd.DataFrame(desc1)
desc1_df

In [ ]:
# Since our personal computers cannot process complex computations on such large datasets,
# we will select 2500 nodes with the highest degree.
# Only edges between these 2500 nodes will be considered.
# Some nodes will be lost out of these 2500 because 
# some of them will only have edges with the nodes not considered in 2500.

sorted_keys = sorted(degree1, key = degree1.get, reverse = True)
selected_nodes = []
for i in range(2500):
    selected_nodes.append(sorted_keys[i])

new_edge_list_temp = []
for i in edge_list:
    if (i[0] in selected_nodes) and (i[1] in selected_nodes):
        new_edge_list_temp.append((i[0], i[1]))
        
G_Facebook_temp = nx.Graph()
G_Facebook_temp.add_edges_from(new_edge_list_temp)

In [ ]:
G_Facebook_nodes = sorted(list(G_Facebook_temp.nodes))

d = {}
count = 0
for i in G_Facebook_nodes:
    d[i] = count
    count += 1
    
new_edge_list = []
for i in new_edge_list_temp:
    new_edge_list.append((d[i[0]], d[i[1]]))
    
G_Facebook = nx.Graph()
G_Facebook.add_edges_from(new_edge_list)

In [ ]:
# Number of nodes and edges

print("The network has " + str(G_Facebook.number_of_nodes()) + " nodes.")
print("The network has " + str(G_Facebook.number_of_edges()) + " edges.")

# Functions

In [ ]:
def get_base_modularity_matrix(network):
    '''
    Obtain the modularity matrix for the whole network

    Parameters
    ----------
    network : nx.Graph or nx.DiGraph
        The network of interest

    Returns
    -------
    np.matrix
        The modularity matrix for `network`

    Raises
    ------
    TypeError
        When the input `network` does not fit either nx.Graph or nx.DiGraph
    '''

    if type(network) == nx.Graph:
        return sparse.csc_matrix(nx.modularity_matrix(network))
    elif type(network) == nx.DiGraph:
        return sparse.csc_matrix(nx.directed_modularity_matrix(network))
    else:
        raise TypeError('Graph type not supported. Use either nx.Graph or nx.Digraph')

def _get_delta_Q(X, a):
    '''
    Calculate the detal modularity
    .. math::
        \deltaQ = s^T \cdot \^{B_{g}} \cdot s
    .. math:: \deltaQ = s^T \cdot \^{B_{g}} \cdot s

    Parameters
    ----------
    X : np.matrix
        B_hat_g
    a : np.matrix
        s, which is the membership vector

    Returns
    -------
    float
        The corresponding :math:`\deltaQ`
    '''

    delta_Q = (a.T.dot(X)).dot(a)

    return delta_Q[0,0]

def get_modularity(network, community_dict):
    '''
    Calculate the modularity. Edge weights are ignored.

    Undirected:
    .. math:: Q = \frac{1}{2m}\sum_{i,j} \(A_ij - \frac{k_i k_j}{2m}\) * \detal_(c_i, c_j)

    Directed:
    .. math:: Q = \frac{1}{m}\sum_{i,j} \(A_ij - \frac{k_i^{in} k_j^{out}}{m}\) * \detal_{c_i, c_j}

    Parameters
    ----------
    network : nx.Graph or nx.DiGraph
        The network of interest
    community_dict : dict
        A dictionary to store the membership of each node
        Key is node and value is community index

    Returns
    -------
    float
        The modularity of `network` given `community_dict`
    '''

    Q = 0
    G = network.copy()
    nx.set_edge_attributes(G, {e:1 for e in G.edges}, 'weight')
    A = nx.to_scipy_sparse_matrix(G).astype(float)

    if type(G) == nx.Graph:
        # for undirected graphs, in and out treated as the same thing
        out_degree = in_degree = dict(nx.degree(G))
        M = 2.*(G.number_of_edges())
        print("Calculating modularity for undirected graph")
    elif type(G) == nx.DiGraph:
        in_degree = dict(G.in_degree())
        out_degree = dict(G.out_degree())
        M = 1.*G.number_of_edges()
        print("Calculating modularity for directed graph")
    else:
        print('Invalid graph type')
        raise TypeError

    nodes = list(G)
    Q = np.sum([A[i,j] - in_degree[nodes[i]]*\
                         out_degree[nodes[j]]/M\
                 for i, j in product(range(len(nodes)),\
                                     range(len(nodes))) \
                if community_dict[nodes[i]] == community_dict[nodes[j]]])
    return Q / M

def get_mod_matrix(network, comm_nodes=None, B=None):
    '''
    This function computes the modularity matrix
    for a specific group in the network.
    (a.k.a., generalized modularity matrix)

    Specifically,
    .. math::
        B^g_{i,j} = B_ij - \delta_{ij} \sum_(k \in g) B_ik
        m = \abs[\Big]{E}
        B_ij = A_ij - \dfrac{k_i k_j}{2m}
        OR...
        B_ij = \(A_ij - \frac{k_i^{in} k_j^{out}}{m}

    When `comm_nodes` is None or all nodes in `network`, this reduces to :math:`B`

    Parameters
    ----------
    network : nx.Graph or nx.DiGraph
        The network of interest
    comm_nodes : iterable (list, np.array, or tuple)
        List of nodes that defines a community
    B : np.matrix
        Modularity matrix of `network`

    Returns
    -------
    np.matrix
        The modularity of `comm_nodes` within `network`
    '''

    if comm_nodes is None:
        comm_nodes = list(network)
        return get_base_modularity_matrix(network)

    if B is None:
        B = get_base_modularity_matrix(network)

    # subset of mod matrix in g
    indices = [list(network).index(u) for u in comm_nodes]
    B_g = B[indices, :][:, indices]
    #print 'Type of `B_g`:', type(B_g)

    # B^g_(i,j) = B_ij - δ_ij * ∑_(k∈g) B_ik
    # i, j ∈ g
    B_hat_g = np.zeros((len(comm_nodes), len(comm_nodes)), dtype=float)

    # ∑_(k∈g) B_ik
    B_g_rowsum = np.asarray(B_g.sum(axis=1))[:, 0]
    if type(network) == nx.Graph:
        B_g_colsum = np.copy(B_g_rowsum)
    elif type(network) == nx.DiGraph:
        B_g_colsum = np.asarray(B_g.sum(axis=0))[0, :]

    for i in range(B_hat_g.shape[0]):
        for j in range(B_hat_g.shape[0]):
            if i == j:
                B_hat_g[i,j] = B_g[i,j] - 0.5 * (B_g_rowsum[i] + B_g_colsum[i])
            else:
                B_hat_g[i,j] = B_g[i,j]

    if type(network) == nx.DiGraph:
        B_hat_g = B_hat_g + B_hat_g.T

    return sparse.csc_matrix(B_hat_g)

def largest_eig(A):
    '''
        A wrapper over `scipy.linalg.eig` to produce
        largest eigval and eigvector for A when A.shape is small
    '''
    vals, vectors = eig(A.todense())
    real_indices = [idx for idx, val in enumerate(vals) if not bool(val.imag)]
    vals = [vals[i].real for i in range(len(real_indices))]
    vectors = [vectors[i] for i in range(len(real_indices))]
    max_idx = np.argsort(vals)[-1]
    return np.asarray([vals[max_idx]]), np.asarray([vectors[max_idx]]).T

In [ ]:
def _divide(network, community_dict, comm_index, B, refine=False):
    '''
    Bisection of a community in `network`.

    Parameters
    ----------
    network : nx.Graph or nx.DiGraph
        The network of interest

    Returns
    -------
    tuple
        If the given community is indivisible, return (None, None)
        If the given community is divisible, return a tuple where
        the 1st element is a node list for the 1st sub-group and
        the 2nd element is a node list for the original group
    '''

    comm_nodes = tuple(u for u in community_dict \
                  if community_dict[u] == comm_index)
    B_hat_g = get_mod_matrix(network, comm_nodes, B)

    # compute the top eigenvector u₁ and β₁
    if B_hat_g.shape[0] < 3:
        beta_s, u_s = largest_eig(B_hat_g)
    else:
        beta_s, u_s = sparse.linalg.eigs(B_hat_g, k=1, which='LR')
    u_1 = u_s[:, 0]
    beta_1 = beta_s[0]
    if beta_1 > 0:
        # divisible
        s = sparse.csc_matrix(np.asmatrix([[1 if u_1_i > 0 else -1] for u_1_i in u_1]))
        if refine:
            improve_modularity(network, comm_nodes, s, B)
        delta_modularity = _get_delta_Q(B_hat_g, s)
        if delta_modularity > 0:
            g1_nodes = np.array([comm_nodes[i] \
                                 for i in range(u_1.shape[0]) \
                                 if s[i,0] > 0])
            #g1 = nx.subgraph(g, g1_nodes)
            if len(g1_nodes) == len(comm_nodes) or len(g1_nodes) == 0:
                # indivisble, return None
                return None, None
            # divisible, return node list for one of the groups
            return g1_nodes, comm_nodes
    # indivisble, return None
    return None, None

def improve_modularity(network, comm_nodes, s, B):
    '''
    Fine tuning of the initial division from `_divide`
    Modify `s` inplace

    Parameters
    ----------
    network : nx.Graph or nx.DiGraph
        The network of interest
    comm_nodes: iterable
        List of nodes for the original group
    s: np.matrix
        A matrix of node membership. Only +1/-1
    B: np.amtrix
        Modularity matrix for `network`
    '''

    # iterate until no increment of Q
    B_hat_g = get_mod_matrix(network, comm_nodes, B)
    while True:
        unmoved = list(comm_nodes)
        # node indices to be moved
        node_indices = np.array([], dtype=int)
        # cumulative improvement after moving
        node_improvement = np.array([], dtype=float)
        # keep moving until none left
        while len(unmoved) > 0:
            # init Q
            Q0 = _get_delta_Q(B_hat_g, s)
            scores = np.zeros(len(unmoved))
            for k_index in range(scores.size):
                k = comm_nodes.index(unmoved[k_index])
                s[k, 0] = -s[k, 0]
                scores[k_index] = _get_delta_Q(B_hat_g, s) - Q0
                s[k, 0] = -s[k, 0]
            _j = np.argmax(scores)
            j = comm_nodes.index(unmoved[_j])
            # move j, which has the largest increase or smallest decrease
            s[j, 0] = -s[j, 0]
            node_indices = np.append(node_indices, j)
            if node_improvement.size < 1:
                node_improvement = np.append(node_improvement, scores[_j])
            else:
                node_improvement = np.append(node_improvement, \
                                        node_improvement[-1]+scores[_j])
            #print len(unmoved), 'max: ', max(scores), node_improvement[-1]
            unmoved.pop(_j)
        # the biggest improvement
        max_index = np.argmax(node_improvement)
        # change all the remaining nodes
        # which are not helping
        for i in range(max_index+1, len(comm_nodes)):
            j = node_indices[i]
            s[j,0] = -s[j, 0]
        # if we swap all the nodes, it is actually doing nothing
        if max_index == len(comm_nodes) - 1:
            delta_modularity = 0
        else:
            delta_modularity = node_improvement[max_index]
        # Stop if ΔQ <= 0 
        if delta_modularity <= 0:
            break

In [ ]:
class CDCGS(nn.Module):
  def __init__(self, A_hat, num_feat, num_hidden):
    super(CDCGS, self).__init__()
    self.num_feat = num_feat
    self.num_hidden = num_hidden
    self.A_hat = A_hat
    self.W_0 = nn.Parameter(torch.ones(num_feat, num_hidden))
    I = F.one_hot(torch.tensor(random.sample(range(0, num_feat), num_feat)), num_classes=num_feat)
    self.I = torch.tensor(I,dtype=torch.float)

  def forward(self, X, A_hat,temp):
    global featureSelector
    global weight_feature
    featureSelector = self.W_0
    results = torch.zeros(self.W_0.size())
    x = 500
    for i in range(x):
        results += F.gumbel_softmax(self.W_0,tau=temp,hard=False)
    weight_feature = results/x
    
    H = torch.mm(torch.mm(self.I,A_hat),self.I.T)
    H = torch.mm(torch.mm(weight_feature.T,A_hat),weight_feature)
    H = torch.div(H, H.sum(axis=0))
    m = nn.Softmax(dim=0)
    return m(H)

In [ ]:
def clustering(G, clusters_number):
    A_hat = nx.adjacency_matrix(G).todense()
    X = np.identity(G.number_of_nodes(), dtype=np.float)

    num_feat = len(G.nodes())
    num_hidden = clusters_number

    model = CDCGS(A_hat, num_feat, num_hidden).to(device)

    def lossFn(output,exp): 
        return torch.sum((torch.diag(-torch.log(output))))

    optimizer = optim.Adam(model.parameters(),lr=1e-2,weight_decay=0,betas=(0.5, 0.999), eps=1e-08)

    A_hat_tensor = torch.Tensor(A_hat).to(device)
    X_tensor = torch.Tensor(X).to(device)

    loss_hist = []
    acc_hist = []
    temp = 4
    logits=[]
    for epoch in range(600):
        model.train()
        model.zero_grad()
        if(epoch == 75):
            temp = 2.5
        elif(epoch == 100):
            temp = 2
        elif(epoch == 150):
            temp = 1.5
        elif(epoch == 200):
            temp = 1
        elif(epoch == 300):
            temp = 0.5
        elif(epoch == 400):
            temp = 0.25
        elif(epoch == 500):
            temp = 0.1
        output = model(X_tensor, A_hat_tensor,temp)
        loss = lossFn(output,torch.diag(torch.ones(output.size()[0])))
        loss_hist.append(loss.item())
        loss.backward()
        optimizer.step()

In [ ]:
def func1(gumbel_matrix, G):
    gumbel_matrix_list = gumbel_matrix.tolist()
    d = {}
    d['node_num'] = []
    d['cluster_num'] = []
    for i in range(G.number_of_nodes()):
        d['node_num'].append(i)
        d['cluster_num'].append(gumbel_matrix_list[i])

    df = pd.DataFrame(d)
    return df

In [ ]:
def func2(gumbel_matrix, G):
    d = {}
    d['node_num'] = []
    d['cluster_num'] = []
    for i in range(G.number_of_nodes()):
        d['node_num'].append(i)
        d['cluster_num'].append(gumbel_matrix[i])
        
    df = pd.DataFrame(d)
    return df

In [ ]:
def cluster_network_analysis(G, gumbel_matrix, df):
    
    temp_D = {}
    temp_D['Threshold Value'] = []
    temp_D['Edges Left in the Network'] = []
    a = set()
    number_of_nodes = G.number_of_nodes()  
    for i in range(number_of_nodes):
        a.add(gumbel_matrix[i])

    b = len(a)
    cluster_matrix = [[0 for i in range(b)] for j in range(b)]
    
    for i in range(G.number_of_edges()):
        if gumbel_matrix[df.iloc[i][0] - 1] != gumbel_matrix[df.iloc[i][1] - 1]:
            cluster_matrix[gumbel_matrix[df.iloc[i][0] - 1]][gumbel_matrix[df.iloc[i][1] - 1]] = \
            cluster_matrix[gumbel_matrix[df.iloc[i][0] - 1]][gumbel_matrix[df.iloc[i][1] - 1]] + 1
            cluster_matrix[gumbel_matrix[df.iloc[i][1] - 1]][gumbel_matrix[df.iloc[i][0] - 1]] = \
            cluster_matrix[gumbel_matrix[df.iloc[i][1] - 1]][gumbel_matrix[df.iloc[i][0] - 1]] + 1

    maximum_connections = max([max(v) for v in cluster_matrix]) 
    print('The maximum number of connections between any two clusters is : ')
    print(maximum_connections)
    print('')
    final_matrix = [[0 for i in range(b)] for j in range(b)]
    for i in range (b):
        for j in range (b):
            final_matrix[i][j] = cluster_matrix[i][j] / maximum_connections

    number_of_connections_left = 0

    temp10 = 0
    variable_matrix10 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix10[i][j] < 0 :
                variable_matrix10[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix10[i][j] != 0:
                #temp10 = temp10 + (cluster_matrix[i][j]*maximum_connections)  
                temp10 = temp10 + cluster_matrix[i][j]

    temp10 = temp10/2             

    '''for i in range (b):
        for j in range (b):
            variable_matrix10[i][j] = variable_matrix10[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0 : ")
    #print(variable_matrix10)
    #print('')
    #print('Number of edges left in the network for threshold value 0 is: ', temp10)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.0)
    temp_D['Edges Left in the Network'].append(temp10)
    
    temp6 = 0
    variable_matrix6 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix6[i][j] < 0.1 :
                variable_matrix6[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix6[i][j] != 0:
                #temp6 = temp6 + (cluster_matrix[i][j]*maximum_connections)  
                temp6 = temp6 + cluster_matrix[i][j]

    temp6 = temp6/2             

    '''for i in range (b):
        for j in range (b):
            variable_matrix6[i][j] = variable_matrix6[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.1 : ")
    #print(variable_matrix6)
    #print('')
    #print('Number of edges left in the network for threshold value 0.1 is: ', temp6)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.1)
    temp_D['Edges Left in the Network'].append(temp6)
    
    temp7 = 0
    variable_matrix7 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix7[i][j] < 0.2 :
                variable_matrix7[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix7[i][j] != 0:
                #temp7 = temp7 + (cluster_matrix[i][j]*maximum_connections)  
                temp7 = temp7 + cluster_matrix[i][j]

    temp7 = temp7/2             

    '''for i in range (b):
        for j in range (b):
            variable_matrix7[i][j] = variable_matrix7[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.2 : ")
    #print(variable_matrix7)
    #print('')
    #print('Number of edges left in the network for threshold value 0.2 is: ', temp7)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.2)
    temp_D['Edges Left in the Network'].append(temp7)
    
    temp4 = 0
    variable_matrix5 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix5[i][j] < 0.3 :
                variable_matrix5[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix5[i][j] != 0:
                #temp4 = temp4 + (cluster_matrix[i][j]*maximum_connections)  
                temp4 = temp4 + cluster_matrix[i][j]

    temp4 = temp4/2             

    '''for i in range (b):
        for j in range (b):
            variable_matrix5[i][j] = variable_matrix5[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.3 : ")
    #print(variable_matrix5)
    #print('')
    #print('Number of edges left in the network for threshold value 0.3 is: ', temp4)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.3)
    temp_D['Edges Left in the Network'].append(temp4)
    
    temp2  = 0

    variable_matrix3 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix3[i][j] < 0.4 :
                variable_matrix3[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix3[i][j] != 0:
                #temp2 = temp2 + (cluster_matrix[i][j]*maximum_connections)
                temp2 = temp2 + cluster_matrix[i][j]

    temp2 = temp2/2            

    '''for i in range (b):
        for j in range (b):
            variable_matrix3[i][j] = variable_matrix3[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.4 : ")
    #print(variable_matrix3)
    #print('')
    #print('Number of edges left in the network for threshold value 0.4 is: ', temp2)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.4)
    temp_D['Edges Left in the Network'].append(temp2)
    
    variable_matrix1 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix1[i][j] < 0.5 :
                variable_matrix1[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix1[i][j] != 0:
                #number_of_connections_left = number_of_connections_left + (cluster_matrix[i][j]*maximum_connections)
                number_of_connections_left = number_of_connections_left + cluster_matrix[i][j]

    temp = 0.5
    temp1 = 0
    number_of_connections_left = number_of_connections_left/2

    '''for i in range (b):
        for j in range (b):
            variable_matrix1[i][j] = variable_matrix1[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.5 : ")
    #print(variable_matrix1)
    #print('')
    #print('Number of edges left in the network for threshold value 0.5 is: ', number_of_connections_left)
    #print('')
    #print('')
    temp_D['Threshold Value'].append(0.5)
    temp_D['Edges Left in the Network'].append(number_of_connections_left)

    variable_matrix2 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix2[i][j] < 0.6 :
                variable_matrix2[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix2[i][j] != 0:
                #temp1 = temp1 + (cluster_matrix[i][j]*maximum_connections)  
                temp1 = temp1 + cluster_matrix[i][j]

    temp1 = temp1/2

    '''for i in range (b):
        for j in range (b):
            variable_matrix2[i][j] = variable_matrix2[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.6 : ")
    #print(variable_matrix2)
    #print('')
    #print('Number of edges left in the network for threshold value 0.6 is: ', temp1)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.6)
    temp_D['Edges Left in the Network'].append(temp1)

    temp3 = 0
    variable_matrix4 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix4[i][j] < 0.7 :
                variable_matrix4[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix4[i][j] != 0:
                #temp3 = temp3 + (cluster_matrix[i][j]*maximum_connections) 
                temp3 = temp3 + cluster_matrix[i][j]

    temp3 = temp3/2       

    ''' for i in range (b):
        for j in range (b):
            variable_matrix4[i][j] = variable_matrix4[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.7 : ")
    #print(variable_matrix4)
    #print('')
    #print('Number of edges left in the network for threshold value 0.7 is: ', temp3)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.7)
    temp_D['Edges Left in the Network'].append(temp3)

    temp8 = 0
    variable_matrix8 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix8[i][j] < 0.8 :
                variable_matrix8[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix8[i][j] != 0:
                #temp8 = temp8 + (cluster_matrix[i][j]*maximum_connections) 
                temp8 = temp8 + cluster_matrix[i][j]

    temp8 = temp8/2       

    ''' for i in range (b):
        for j in range (b):
            variable_matrix8[i][j] = variable_matrix8[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.8 : ")
    #print(variable_matrix8)
    #print('')
    #print('Number of edges left in the network for threshold value 0.8 is: ', temp8)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.8)
    temp_D['Edges Left in the Network'].append(temp8)

    
    temp9 = 0
    variable_matrix9 = final_matrix
    for i in range (b):
        for j in range (b):
            if variable_matrix9[i][j] < 0.9 :
                variable_matrix9[i][j] = 0

    for i in range (b):
        for j in range (b):
            if variable_matrix9[i][j] != 0:
                #temp9 = temp9 + (cluster_matrix[i][j]*maximum_connections) 
                temp9 = temp9 + cluster_matrix[i][j]

    temp9 = temp9/2       

    ''' for i in range (b):
        for j in range (b):
            variable_matrix9[i][j] = variable_matrix9[i][j]*maximum_connections'''
    
    #print("the adjacency matrix of the clusters assuming that the threshold value is 0.9 : ")
    #print(variable_matrix9)
    #print('')
    #print('Number of edges left in the network for threshold value 0.9 is: ', temp9)
    #print('')
    #print('')
    
    temp_D['Threshold Value'].append(0.9)
    temp_D['Edges Left in the Network'].append(temp9)
    
    df = pd.DataFrame(temp_D)
    return df

# Section 2: Normal Clustering

### Zachary's Karate Club Dataset

In [ ]:
clusters_number = 2

In [ ]:
# Running the Clustering

clustering(G, clusters_number)

In [ ]:
gumbel_matrix_karate = weight_feature.detach().max(dim=1)[1]
labels_pred = gumbel_matrix_karate.data.numpy()

print('Modularity for clustering in ' + str(clusters_number) + ' clusters is ' + str(get_modularity(G, labels_pred)) + '.')

In [ ]:
# Creating a DataFrame to store data about nodes and the cluster they belong to
# Clusters: 0, 1

df = func1(gumbel_matrix_karate, G)
df.head()

In [ ]:
# Size of each cluster

df.groupby('cluster_num').size()

### Facebook Dataset

In [ ]:
clusters_number = 10

In [ ]:
# Running the Clustering

clustering(G_Facebook, clusters_number)

In [ ]:
gumbel_matrix_facebook = weight_feature.detach().max(dim=1)[1]
labels_pred = gumbel_matrix_facebook.data.numpy()

print('Modularity for clustering in ' + str(clusters_number) + ' clusters is ' + str(get_modularity(G_Facebook, labels_pred)) + '.')

In [ ]:
# Creating a DataFrame to store data about nodes and the cluster they belong to
# Clusters: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9

df = func1(gumbel_matrix_facebook, G_Facebook)
df.head()

In [ ]:
# Size of each cluster

df.groupby('cluster_num').size()

# Section 3: Custom Threshold Clustering (Multiple Clustering)

- Each node has a certain probability of belonging to a cluster.
- In Section 2, for whichever cluster this probability was maximum, the node was assigned to that cluster.
- Here, we assign all the clusters to a node for which the probability is more than or equal to a given threshold (threshold given by user).

### Zachary's Karate Club Dataset

In [ ]:
# These values can be changed by the user

clusters_number = 5
prob_threshold = 0.1

In [ ]:
# Running the Clustering

clustering(G, clusters_number)

In [ ]:
gumbel_matrix = []

for i in weight_feature.tolist():
    node_cluster = []
    for j in range(clusters_number):
        if i[j] >= prob_threshold:
            node_cluster.append(j)
    gumbel_matrix.append(node_cluster)

In [ ]:
# Creating a DataFrame to store data about nodes and the cluster they belong to
# Clusters: 0, 1, 2, 3, 4
    
df = func2(gumbel_matrix, G)
df.head()

In [ ]:
# Count of nodes assigned to more than 1 cluster

count = 0
for i in df['cluster_num']:
    if len(i) > 1:
        count += 1
        
print(str(count) + " nodes got assigned to more than 1 cluster.")

### Facebook Dataset

In [ ]:
# These values can be changed by the user

clusters_number = 50
prob_threshold = 0.1

In [ ]:
# Running the Clustering

clustering(G_Facebook, clusters_number)

In [ ]:
gumbel_matrix = []

for i in weight_feature.tolist():
    node_cluster = []
    for j in range(clusters_number):
        if i[j] >= prob_threshold:
            node_cluster.append(j)
    gumbel_matrix.append(node_cluster)

In [ ]:
# Creating a DataFrame to store data about nodes and the cluster they belong to

df = func2(gumbel_matrix, G_Facebook)
df.head()

In [ ]:
# Count of nodes assigned to more than 1 cluster

count = 0
for i in df['cluster_num']:
    if len(i) > 1:
        count += 1

print(str(count) + " nodes got assigned to more than 1 cluster.")

# Section 4: Determining Ideal Number of Clusters

- We plot a Modularity vs. Number of Clusters (clusters_number) graph for a fixed range of number of clusters we want.
- The value of clusters_number for which the modularity is maximum is the ideal number of clusters.

### Zachary's Karate Club Dataset

In [ ]:
# Maintaining a list to save the modularity values

modularity_list = []

In [ ]:
# Run a loop for different number of clusters.
# Range of clusters_number: [2, 11)

for i in range(2, 11):
   clustering(G, i)

   gumbel_matrix = weight_feature.detach().max(dim=1)[1]
   labels_pred = gumbel_matrix.data.numpy()
   modularity_list.append((i, get_modularity(G, labels_pred)))

In [ ]:
xAxis = []
yAxis = []
d = {}
for i in modularity_list:
    xAxis.append(i[0])
    yAxis.append(i[1])
    d[i[1]] = i[0]
    
plt.plot(np.array(xAxis), np.array(yAxis))

In [ ]:
print("Maximum modularity in this range = ", max(yAxis))
print("Which is achieved for clusters_number = ", d[max(yAxis)])

### Facebook Dataset

In [ ]:
# Maintaining a list to save the modularity values

modularity_list = []

In [ ]:
# Run a loop for different number of clusters.
# Range of clusters_number: [10, 101)

for i in range(10, 101):
    clustering(G_Facebook, i)

    gumbel_matrix = weight_feature.detach().max(dim=1)[1]
    labels_pred = gumbel_matrix.data.numpy()
    modularity_list.append((i, get_modularity(G_Facebook, labels_pred)))

In [ ]:
xAxis = []
yAxis = []
d = {}
for i in modularity_list:
    xAxis.append(i[0])
    yAxis.append(i[1])
    d[i[1]] = i[0]
    
plt.plot(np.array(xAxis), np.array(yAxis))

In [ ]:
print("Maximum modularity in this range = ", max(yAxis))
print("Which is achieved for clusters_number = ", d[max(yAxis)])

# Section 5: Cluster Network Analysis

### Zachary's Karate Club Dataset

In [ ]:
network0 = pd.read_csv(r"karate_club.csv")
edge_list = []
for i in range(78):
    edge_list.append((network0.iloc[i]['node_1'], network0.iloc[i]['node_2']))
    
cluster_network_analysis(G, gumbel_matrix_karate.tolist(), network0)

### Facebook Dataset

In [ ]:
d_edge = {}
d_edge['node_1'] = []
d_edge['node_2'] = []

for i in new_edge_list:
    d_edge['node_1'].append(i[0])
    d_edge['node_2'].append(i[1])

network1 = pd.DataFrame(d_edge)

cluster_network_analysis(G_Facebook, gumbel_matrix_facebook, network1)